In [ ]:
# ActionChains Documentation: 
# https://www.selenium.dev/selenium/docs/api/py/webdriver/selenium.webdriver.common.action_chains.html#selenium.webdriver.common.action_chains.ActionChains.release

# Beautiful Soup Docs: https://www.crummy.com/software/BeautifulSoup/bs4/doc/


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# Set up Selenium
options = webdriver.ChromeOptions()
options.headless = True  # Run Chrome in headless mode (without GUI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Visit Vivino's page
url = "https://www.vivino.com/explore"
driver.get(url)

# Wait for the page to load properly
wait = WebDriverWait(driver, 10)

# Step 1: click the dropdown menu
dropdown = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "simpleLabel-module__selectedKey--3ngzL")))
dropdown.click()
time.sleep(2)

# Step 2: select "United States" from the list
country_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@data-value='us']")))
country_option.click()
time.sleep(3)

# Step 3: unselect the "Red" wine filter
red_wine_label = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'MuiButtonBase-root') and @data-testid='wineTypes_1']")))
driver.execute_script("arguments[0].click();", red_wine_label)
time.sleep(3)

# Step 4: drag slider to select all prices.
# Find the first slider handle (left)
slider_1 = driver.find_element(By.XPATH, "//div[@class='rc-slider-handle rc-slider-handle-1']")

# Use ActionChains to drag the first slider
actions = ActionChains(driver)
actions.click_and_hold(slider_1).move_by_offset(-10, 0).release().perform()

# Find the second slider handle (right)
slider_2 = driver.find_element(By.XPATH, "//div[@class='rc-slider-handle rc-slider-handle-2']")

# Use ActionChains to drag the second slider
actions = ActionChains(driver)
actions.click_and_hold(slider_2).move_by_offset(150, 0).release().perform()

# Step 5: Select Any Rating option
any_rating_label = wait.until(EC.element_to_be_clickable((By.XPATH, "//label[.//input[@name='rating' and @value='1']]")))
driver.execute_script("arguments[0].click();", any_rating_label)


# Step 6: Select Countries
countries_dropdown = driver.find_element(By.XPATH, "//legend[@data-testid='filter-toggle-button'][.//h5[.='Countries']]")
driver.execute_script("arguments[0].click();", countries_dropdown)
time.sleep(2)
countries_ar = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'MuiButtonBase-root')]/parent::label[@data-testid='countries_ar']")))
#countries_cl = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'MuiButtonBase-root')]/parent::label[@data-testid='countries_cl']")))
#driver.execute_script("arguments[0].click(); arguments[1].click();", countries_ar countries_cl)
driver.execute_script("arguments[0].click();", countries_ar)
countries_search = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@id='filter-checkboxes-search' and @placeholder='Search countries']")))
countries_search.click()
countries_search.send_keys("Brazil")
time.sleep(1)
countries_br = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'MuiButtonBase-root')]/parent::label[@data-testid='countries_br']")))
driver.execute_script("arguments[0].click()", countries_br)

# Wait and see the result
time.sleep(5)

def getPageLinks(list):
    next_page = True

    # Get the page source after it's fully loaded
    html = driver.page_source

    # Pass it to BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    links = [a["href"] for a in soup.find_all('a', {"data-testid": "vintagePageLink"})]

    full_links = ["https://www.vivino.com" + link for link in links]

    list.extend(full_links)

        
    try:
        next_page_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@data-trackingid='buttonDefault'][.//span[.='Next']]")))
        
        try:
            disabled_btn = next_page_btn.get_attribute("aria-disabled")        
            if disabled_btn == "true":
                next_page = False
                return next_page
        except:
            driver.execute_script("arguments[0].click()", next_page_btn)
            time.sleep(5)
            next_page = False
            return next_page
        
    except:
        next_page = False
        return next_page

link_list = []
next_page_available = True

while next_page_available:
    next_page_available = getPageLinks(link_list)

link_df = pd.DataFrame(link_list, columns=["wine_link"])
link_df.to_csv("wine_links.csv", index=False)

driver.quit()

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

csv = pd.read_csv("wine_test.csv")
wine_df = pd.DataFrame(csv, columns=["wine_link"]).drop_duplicates()
wine_df["name"] = None
wine_df["year"] = None
wine_df["winery"] = None
wine_df["rating"] = None
wine_df["rating_qty"] = None
wine_df["price"] = None
wine_df["body"] = None
wine_df["tannis"] = None
wine_df["sweetness"] = None
wine_df["acidity"] = None
wine_df["notes"] = None
wine_df["pairings"] = None
wine_df["grapes"] = None
wine_df["region"] = None
wine_df["style"] = None
wine_df["image"] = None

options = webdriver.ChromeOptions()
options.headless = False  
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Iterar en las columnas del DF
for index, row in wine_df.iterrows():
    link = row["wine_link"]
    driver.get(link)
    time.sleep(3)  

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # DATA 1: NOMBRE (name)
    wine_name_tag = soup.find("a", {"data-cartitemsource": "wine-page-master-link"})
    wine_name = wine_name_tag.get_text(strip=True) if wine_name_tag else None

    # DATA 2: AÑO (year)
    wine_year = None
    if wine_name_tag:
        parent_div = wine_name_tag.find_parent("div")
        if parent_div:
            year_text = parent_div.get_text(strip=True)
            wine_year = year_text.replace(wine_name, "").strip()

    # DATA 3: BODEGA (WINERY)
    winery = soup.find("h1").find("div").find("a").find("div").get_text(strip=True)

    # DATA 4: REVIEWS (Rating)
    rating_tag = soup.find("a", {"href": "#all_reviews"})
    rating_value = rating_tag.find("div").find("div").get_text(strip=True)
    rating_qty = rating_tag.find_all("div")[-1].get_text(strip=True).split(" ")[0]

    # DATA 5: PRICE (Precio)
    price = soup.find("span", {"class": "purchaseAvailability__currentPrice--3mO4u"}).get_text(strip=True).replace("$", "")

    # DATA 6: TASTE (Sabor)

    wine_taste = soup.find_all("tr", {"class": "tasteStructure__tasteCharacteristic--jLtsE"})

    body = None
    tannis = None
    sweetness = None
    acidity = None

    for i in wine_taste:
        # name = buscar sabor
        name = wine_taste.find("td").get_text(strip=True)
        bar = wine_taste[i].find("span", {"class", "indicatorBar__progress--3aXLX"})["style"]
        val = bar.split(";")[1].split(":")[1]
        
         # 6.1 : Light/Bold - Body (Cuerpo)   
        if name == "Light":
            body = val
        
        # 6.2 : Smooth/Tannic - Tannis (Tanicidad)
        elif name == "Smooth":
            tannis = val
        
        # 6.3 : Dry/Sweet - Sweetness (Dulzura)
        elif name == "Dry":
            sweetness = val
        
        # 6.4 : Soft/Acid - Acidity (Acidez)
        elif name == "Soft":
            acidity = val


    # DATA 7 : NOTE MENTIONS (Menciones de notas)

    mentions = soup.find_all("div", {"data-testid": "mentions"})

    mentions_dict = {}

    for mention in mentions:
        note = mention.find("span").get_text(strip=True)
        n_mentions = mention.get_text().split(" ")[0]
        mentions_dict[str(note)] = int(n_mentions)

    
    # DATA 8 : PAIRINGS (Maridajes)

    pairing_tags = soup.find("div", {"class": "foodPairing__foodContainer--1bvxM"}).find_all("a", recursive=False)
    
    pairing_list = []

    for i in pairing_tags:
        pairing = i.find("div", {"role": "img"})["aria-label"].lower()
        pairing_list.append(pairing)


    # DATA 9 : GRAPES (Uvas)
    wine_facts = soup.find_all("tr", {"data-testid": "wineFactRow"})

    grapes = list(wine_facts[1].find("td").get_text(strip=True).split(","))

    # DATA 10 : REGION
    region = list(wine_facts[2].find("td").get_text(strip=True).split("/"))

    # DATA 11 : STYLE (Estilo)
    style = wine_facts[3].find("td").get_text()

    # DATA 12 : WINE IMAGE (Imagen del Vino)
    img = "https:" + soup.find("img", {"class": "wineLabel-module__image--3HOnd"})["src"]

    # ÚLTIMO PASO: GUARDAR EN DATAFRAME
    wine_df.at[index, "name"] = wine_name
    wine_df.at[index, "year"] = wine_year
    wine_df.at[index, "winery"] = winery
    wine_df.at[index, "rating"] = rating_value
    wine_df.at[index, "rating_qty"] = rating_qty
    wine_df.at[index, "price"] = price
    wine_df.at[index, "body"] = body
    wine_df.at[index, "tannis"] = tannis
    wine_df.at[index, "sweetness"] = sweetness
    wine_df.at[index, "acidity"] = acidity
    wine_df.at[index, "notes"] = mentions_dict
    wine_df.at[index, "pairings"] = pairing_list
    wine_df.at[index, "grapes"] = grapes
    wine_df.at[index, "region"] = region
    wine_df.at[index, "style"] = style
    wine_df.at[index, "image"] = img

# Guardar el CSV actualizado
wine_df.to_csv("updated_test.csv", index=False)

driver.quit()

wine_df

AttributeError: 'NoneType' object has no attribute 'get_text'

In [8]:
wine_df.dropna()

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,notes,pairings,grapes,region,style,image
0,https://www.vivino.com/US/en/luigi-bosca-parai...,Paraiso,2020,Luigi Bosca,4.8,575,164.99,73.43435%,50.905894999999994%,13.619732499999998%,44.74824%,"{'black fruit': 14, 'oaky': 13, 'earthy': 5, '...","[beef, lamb, poultry, game (deer, venison)]","[Cabernet Sauvignon, Malbec]","[Argentina, Mendoza]",Argentinian Cabernet Sauvignon - Malbec,https://images.vivino.com/thumbs/_Bf6JTwYRpSX6...
1,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2015,Catena Zapata,4.7,295,675,74.09986749999999%,55.86741500000001%,14.329525%,54.65305%,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","[beef, lamb, poultry, pasta]","[Cabernet Sauvignon, Malbec, Merlot]","[Argentina, Mendoza, Agrelo]",Argentinian Bordeaux Blend,https://images.vivino.com/thumbs/Yt464jw0QS-ug...
2,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2017,Catena Zapata,4.7,218,580,74.09986749999999%,55.86741500000001%,14.329525%,54.65305%,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","[beef, lamb, poultry, pasta]","[Cabernet Sauvignon, Malbec, Merlot]","[Argentina, Mendoza, Agrelo]",Argentinian Bordeaux Blend,https://images.vivino.com/thumbs/Yt464jw0QS-ug...
3,https://www.vivino.com/US/en/monteviejo-la-vio...,La Violeta,2013,Monteviejo,4.7,202,150,73.06289000000001%,40.9165975%,21.4814425%,36.61685%,"{'oaky': 51, 'black fruit': 49, 'earthy': 26, ...","[beef, lamb]",[100%Malbec],"[Argentina, Mendoza, Uco Valley]",Argentinian Uco Valley Malbec Red,https://images.vivino.com/thumbs/FO-x9h3mQHSx9...
4,https://www.vivino.com/US/en/vina-cobos-cobos-...,Cobos Volturno,2018,Viña Cobos,4.7,198,399.99,71.05489250000001%,45.630624999999995%,24.2544575%,38.45142%,"{'oaky': 86, 'black fruit': 49, 'spices': 17, ...","[beef, lamb, poultry, game (deer, venison)]",[100%Cabernet Sauvignon],"[Argentina, Mendoza, Perdriel]",Argentinian Cabernet Sauvignon - Malbec,https://images.vivino.com/thumbs/D1Mf1fYnRnage...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,https://www.vivino.com/vina-cobos-bramare-chan...,Bramare Chañares Estates Cabernet Franc,2016,Viña Cobos,4.4,550,99.99,54.938542500000004%,57.920042499999994%,12.821594999999999%,52.62113%,"{'oaky': 39, 'black fruit': 26, 'spices': 15, ...","[beef, lamb, pork, vegetarian, goat's milk che...",[100%Cabernet Franc],"[Argentina, Mendoza, Uco Valley]",Argentinian Uco Valley Cabernet Franc Red,https://images.vivino.com/thumbs/f82x6OBxQnOyd...
77,https://www.vivino.com/vina-cobos-bramare-malb...,Bramare Malbec Uco Valley,2022,Viña Cobos,4.4,509,38.84,68.08894249999999%,36.988890000000005%,22.579679999999996%,35.052262500000005%,"{'oaky': 771, 'black fruit': 609, 'earthy': 24...","[beef, lamb]",[Malbec],"[Argentina, Mendoza, Uco Valley]",Argentinian Uco Valley Malbec Red,https://images.vivino.com/thumbs/yzinnOvTSO28P...
78,https://www.vivino.com/zuccardi-finca-piedra-i...,Finca Piedra Infinita Malbec,2018,Zuccardi,4.4,434,339,65.89438750000001%,43.3240175%,20.397017499999997%,46.79042%,"{'black fruit': 188, 'earthy': 122, 'oaky': 12...","[beef, lamb]",[Malbec],"[Argentina, Mendoza, Uco Valley, Paraje Altamira]",Argentinian Uco Valley Malbec Red,https://images.vivino.com/thumbs/WuH10JZKRHeUp...
79,https://www.vivino.com/catena-zapata-nicasia-v...,Nicasia Vineyard Altamira Malbec,2020,Catena Zapata,4.4,382,99,67.59591499999999%,29.926800000000007%,21.9947375%,34.0994475%,"{'oaky': 314, 'black fruit': 297, 'earthy': 12...","[beef, lamb]",[100%Malbec],"[Argentina, Mendoza, Uco Valley, Paraje Altamira]",Argentinian Uco Valley Malbec Red,https://images.vivino.com/thumbs/P9umSXr8QFGJd...
